<a href="https://colab.research.google.com/github/JMQuehl/stable-diffusion/blob/colab/colab_notebooks/StableDiffusionUI_Voldemort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Adapted from: https://colab.research.google.com/drive/1AfAmwLMd_Vx33O9IwY2TmO9wKZ8ABRRa

In [ ]:
!nvidia-smi

In [ ]:
import os
os.kill(os.getpid(), 9) # This will crash Colab (required, everything will still be intact so dont worry)

Clone webui repository

In [ ]:
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui
%cd stable-diffusion-webui

Install requirements, download repositories of stable diffusion

In [ ]:
!pip install -r requirements.txt
!mkdir repositories
!git clone https://github.com/CompVis/stable-diffusion.git repositories/stable-diffusion
!git clone https://github.com/CompVis/taming-transformers.git repositories/taming-transformers
!git clone https://github.com/sczhou/CodeFormer.git repositories/CodeFormer
!pip install -r repositories/CodeFormer/requirements.txt 

Download the model from huggingface. See link at the start for different ways.

In [ ]:
use_gdrive = True #@param{type: 'boolean'}
save_missing_models_to_gdrive = True #@param{type: 'boolean'}
force_remount = False

if use_gdrive:
  from google.colab import drive
  gdrive_models_folder = "/content/drive/MyDrive/AI/models" #@param{type: 'string'}
  drive.mount("/content/drive",force_remount=force_remount)

import os
full_gdrive_sd_path = os.path.join(gdrive_models_folder, "sd-v1-4.ckpt")
if not use_gdrive or not os.path.isfile(full_gdrive_sd_path):
  #@title Normal 1.4 model
  # get a token from https://huggingface.co/settings/tokens
  user_token = "get a token from https://huggingface.co/settings/tokens" #@param {type:"string"}
  user_header = f"\"Authorization: Bearer {user_token}\""
  !wget --header={user_header} https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt -O model.ckpt
  if use_gdrive and save_missing_models_to_gdrive:
    !cp model.ckpt {full_gdrive_sd_path}
else:
  !cp {full_gdrive_sd_path} model.ckpt

full_gdrive_gfpgan_path = os.path.join(gdrive_models_folder, "GFPGANv1.3.pth")
if not use_gdrive or not os.path.isfile(full_gdrive_gfpgan_path):
  !wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth
  if use_gdrive and save_missing_models_to_gdrive:
    !cp GFPGANv1.3.pth {full_gdrive_gfpgan_path}
else:
  !cp {full_gdrive_gfpgan_path} GFPGANv1.3.pth
  

Set standard-values for web-ui for easy use with gdrive.

In [ ]:
import json
if use_gdrive:
  gdrive_txt2img_out_folder = "/content/drive/MyDrive/AI/StableDiffusion/txt2img_out" #@param{type: 'string'}
  gdrive_img2img_out_folder = "/content/drive/MyDrive/AI/StableDiffusion/img2img_out" #@param{type: 'string'}
  gdrive_extras_out_folder = "/content/drive/MyDrive/AI/StableDiffusion/extras_out" #@param{type: 'string'}
  gdrive_grids_out_folder = "/content/drive/MyDrive/AI/StableDiffusion/grids_out" #@param{type: 'string'}
  save_to_sub_dirs = True #@param{type: 'boolean'}
  sub_dir_name_length = 10 #@param{type: 'slider', min:1, max:32, step:1}

  class OptionInfo:
    def __init__(self, default=None, label="", component=None, component_args=None):
        self.default = default
        self.label = label
        self.component = component
        self.component_args = component_args

  optsdata = {
      "outdir_samples": "",
      "outdir_txt2img_samples": gdrive_txt2img_out_folder,
      "outdir_img2img_samples": gdrive_img2img_out_folder,
      "outdir_extras_samples": gdrive_extras_out_folder,
      "outdir_grids": gdrive_grids_out_folder, 
      "outdir_txt2img_grids": gdrive_grids_out_folder,
      "outdir_img2img_grids": gdrive_grids_out_folder,
      "save_to_dirs": save_to_sub_dirs,
      "save_to_dirs_prompt_len": sub_dir_name_length,
      "outdir_save": "log/images",
      "samples_save": True,
      "samples_format": 'png', 
      "grid_save": True, 
      "return_grid": True, 
      "grid_format": 'png', 
      "grid_extended_filename": False, 
      "grid_only_if_multiple": True, 
      "n_rows": -1, 
      "jpeg_quality": 80, 
      "export_for_4chan": True, 
      "enable_pnginfo": True, 
      "font": "", 
      "enable_emphasis": True, 
      "save_txt": False, 
      "ESRGAN_tile": 192, 
      "ESRGAN_tile_overlap": 8, 
      "random_artist_categories": [],
      "upscale_at_full_resolution_padding": 16, 
      "show_progressbar": True,
      "show_progress_every_n_steps": 0, 
      "multiple_tqdm": True, 
      "face_restoration_model": None,
      "code_former_weight": 0.5
      }
  with open("config.json", "w", encoding="utf8") as file:
    json.dump(optsdata, file)


Kill the runtime.

In [ ]:
import os
os.kill(os.getpid(), 9) # This will crash Colab (required, everything will still be intact so dont worry)

Change into Web UI directory and download updates

In [ ]:
%cd stable-diffusion-webui
!git pull

Launch web ui. You will get a link to nnn.gradio.app, follow it.

In [ ]:
import sys
sys.argv = ["--opt-split-attention"]

import webui
import modules.ui
import modules.txt2img

import os

# set opts.outdir_samples / opts.outdir_txt2img_samples, etc.

demo = modules.ui.create_ui(
	txt2img=webui.wrap_gradio_gpu_call(modules.txt2img.txt2img),
	img2img=webui.wrap_gradio_gpu_call(modules.img2img.img2img),
	run_extras=webui.wrap_gradio_gpu_call(webui.run_extras),
	run_pnginfo=webui.run_pnginfo
)
debug_output_in_colab = True #@param{type: 'boolean'}
demo.launch(share=True, debug=debug_output_in_colab)
